# Generative Adversarial Nets

- **GAN(Generative Adversarial Nets)**: 생성자(Generator)와 판별자(Discriminator) 모델이 경쟁하면서 학습하는 모델로, 실제 데이터와 유사한 데이터를 생성해내는 모델을 만드는 것을 목표로 학습한다.

# 0. Abstract

- **생성자 모델(Generative Model) G**는 실제 데이터 분포를 따라하는 방향으로 학습한다.
- **판별자 모델(Discriminative Model) D**는 생성자가 생성한 데이터가 아닌 실제 데이터일 확률을 추정한다.
- 생성자 G는 판별자 D가 실수할 확률을 최대화하는 방향으로 학습한다.
- 최종적으로 생성자 G는 실제와 유사한 데이터를 생성하고, 판별자 D는 실제 데이터와 생성된 데이터를 판별할 확률이 1/2가 되도록 한다.

# 1. Introduction

- **분류 모델(Discriminative Model)**은 딥러닝에서 크게 성공했다.
    - 이는 조각적으로 선형인 활성화함수(piecewise linear unit)를 이용한 역전파(backpropagation)과 드롭아웃(dropout algorithm)이 기반이 되었기 때문이다.

- 그러나 **생성 모델(Generative Model)**은 비교적 크게 성공하지 못했다.
    - 그 이유는 다음과 같다.
        1. 최대우도추정(maximum likelihood estimation)의 확률적인 계산이 쉽지 않다.
        2. 조각적으로 선형인 활성화함수(piecewise linear unit)의 이점을 활용하기도 쉽지 않다.
        
- 이 논문에서는 생성 모델의 어려움을 회피한 GAN 모델을 제안한다.

- 생성자 모델(Generative Model)은 지폐 위조범으로, 판별자 모델(Discriminative Model)은 위조지폐를 판별하는 경찰로 비유할 수 있다.
    - 판별자가 위조지폐와 진짜를 구분할 수 없을 때까지 학습시킨다.

- 제안하는 체계(framework)는 현존하는 여러 모델을 적용하여 다양한 모델로 구성할 수 있다.

- 이 논문에서는 **Adversarial Model**을 다룬다.
    - **Adversarial Model**: 생성자에서 **random noise**를 입력하여 데이터를 생성하고, 생성자와 판별자에 모두 **다층 퍼셉트론(multilayer perceptron)**을 이용하는 모델

- 해당 모델은 역전파(backpropagation), 드롭아웃(dropout algorithm)만을 이용하여 학습이 가능하다.

<img src="00.png" width=70% height=70%>

# 2. Adversarial nets

- 표기는 다음을 따르자.
    - $p_{g}$: 생성자의 분포(generator's distribution)
    - $p_{data}$: 실제 데이터의 분포
    - $p_{z}$: 생성자에 입력되는 잡음의 사전 분포(prior distribution of input noise variables)
    - $\theta_g$: 생성자의 파라미터(parameters of generator)
    - $\theta_d$: 판별자의 파라미터(parameters of discriminator)
    - $G(z;\theta_g)$: 생성자 함수(generator function) (multilayer perceptron에 의해 미분가능하다.)
    - $D(x;\theta_d)$: 판별자 함수(discriminator function) (multilayer perceptron에 의해 미분가능하다.)
        - 판별자는 실제 데이터는 1, 생성된 데이터는 0으로 판별하도록 확률을 출력한다.

- 생성자 G와 판별자 D는 다음과 같은 $V(G, D)$ 값에 대한 minimax 게임을 진행한다.
$$
\min_{G} \max_{D} V(G, D) = \mathbb{E}_{\mathbf{x}\sim p_{data}(\mathbf{x})} [\log D(\mathbf{x})] + \mathbb{E}_{\mathbf{z}\sim p_{\mathbf{z}}(\mathbf{z})} [\log (1-D(G(\mathbf{z})))]
$$
    - 생성자 G는 $D(G(z))=1$이 되길 원한다.
        - 생성자는 $V$의 첫 번째 항에는 영향을 줄 수 없고, 두 번째 항에서 $1-D(G)=0$이 되도록 학습해야 하므로 $V$를 최소화해야 한다. ($-\infty$가 되도록 학습)
    - 판별자 D는 $D(x)=1, D(G(z))=0$이 되길 원한다.
        - 판별자는 $V$의 첫 번째 항에서 $D(x)=1$이 되도록 학습하고, 두 번째 항에서 $1-D(G(z))=1$이 되도록 학습해야 하므로 $V$를 최대화해야 한다. ($0$이 되도록 학습)

- 학습은 다음과 같은 과정으로 이루어진다.
    1. 판별자 D를 k번 최적화한다.
    2. 생성자 G를 1번 최적화한다.
    3. 1, 2를 계속 반복한다.
    - 이러한 학습 방법은 판별자 D를 완전히 최적화시키는 것보다  
     1) 계산량이 적어지며 2) 유한한 데이터셋으로 인한 오버피팅(overfitting)이 일어날 가능성도 적어진다.

- 위의 학습 알고리즘은 다음과 같이 나타낼 수 있다.
<img src="01.png" width=70% height=70%>

- 학습 과정을 아주 간단한 그림으로 나타내면 다음과 같다.
<img src="02.png" width=100% height=100%>

- 생성자 G를 학습시킬 때에는 $\log(1-D(G(z)))$를 최소화하는 것보다 $\log D(G(z))$를 최대화하는 것이 좋다.
    - 만약 G의 성능이 나쁘다면, $\log(1-D(G(z)))$에서는 gradient가 낮아서 매우 느리게 학습하는 반면,  
    $\log(D(G(z)))$에서는 gradient가 높아서 초기에 빠르게 학습할 수 있다.

<img src="03.png" width=50% height=50%>

# 3. Theoretical Results

## 3.1. Background

- **Cross Entropy**
$$
CE = \sum_{x} (-p(x) \log q(x))
$$
    - 만약 $p(x)$의 값이 0 또는 1이라면, 최솟값은 $p(x)=q(x)$일 때 0이고, 최댓값은 $\infty$로 존재하지 않는다.
    - 만약 $p(x)$의 값들 중 0 또는 1이 없다면, 최솟값은 $p(x)=q(x)$일 때 $CE = \sum_{x} (-p(x) \log p(x))$으로, 0보다 크다. (이는 Gibb's inequality으로부터 증명된다.)

- **KL divergence**
$$
KL(p||q) = \sum_{x} p(x) \log \frac{p(x)}{q(x)}
$$
    - $KL(p||q) = \sum_{x} p(x) \log \frac{p(x)}{q(x)} = - \sum_{x} p(x) \log q(x) + \sum_{x} p(x) \log p(x)$ 이므로 최솟값은 $p(x)=q(x)$일 때 0으로 맞춰준 것이다.
    - 두 분포 $p, q$가 다른 정도를 보여준다. 두 분포의 차이가 클수록 KL divergence 값이 커진다.
    - 그러나 $KL(p||q)$와 $KL(q||p)$의 값이 다르다는 문제점이 있다.

- **Jensen-Shannon divergence**
$$
JSD(p||q) = \frac{1}{2} KL(p||\frac{p+q}{2}) + \frac{1}{2} KL(q||\frac{p+q}{2})
$$
    - 최솟값은 $p(x)=q(x)$일 때 0이고, 최댓값은 $\log 2$이다.
    - $JSD(p||q)=JSD(q||p)$이므로 대칭적이다.
    - JSD는 두 분포 $p, q$의 차이를 측정하는 거리 지표로 사용할 수 있다.

## 3.2. Propositions and Theorem

- **Proposition 1**: 판별자 D를 완전히 최적화한다면 어떤 값으로 수렴하는지 확인해보자.
<img src="04.jpg" width=100% height=100%>

- **Theorem 1**: 생성자 G가 $V$를 최소화하려면 $p_g$가 어떤 분포여야 하는지 알아보자.
<img src="05.jpg" width=100% height=100%>

- **Proposition 2**: 실제로 위에서 제시한 Algorithm을 이용하면 $p_g$가 $p_{data}$로 수렴하는지 확인해보자.
<img src="06.jpg" width=100% height=100%>

- 실제로는 이론과 달리 multilayer perceptron을 사용하므로 그 특성상 임계점(critical point)가 많다.
    - 그러나 이론적인 보장이 부족함에도 불구하고 학습이 잘 되는 모습을 보인다.

# 4. Advantages and disadvantages

- **Disadvantages**
1. $p_{g}(x)$가 명확하게 표현되지 않는다.
2. 학습하는 동안 G와 D가 서로 동기화되어야 한다.
    - **모드 붕괴(Mode collapse)**: 생성 모델에서 다양한 출력 대신 일부분의 출력만을 계속 생성해내는 현상
        - 이는 D가 제대로 최적화되지 않은 상태에서 G를 학습시키면 종종 발생한다.
        - 이를 해결하기 위해 D를 최적화시키는 데에 더 많은 학습 step을 할애하거나, 다양한 모델 기법을 적용하여 해결할 수 있다.
<img src="07.png" width=100% height=100%>

- **Advantages**
    1. Markov Chains 같은 복잡한 계산이 포함된 방법을 사용하지 않고, 오로지 Backpropagation만을 이용한다.
    2. 다양한 함수를 해당 모델에 적용해볼 수 있다.
    3. 기존보다 더 정교한 분포를 학습할 수 있다.
    4. 조건을 추가하여 Conditional Generative Model로 확장할 수 있다.

# 5. How to evaluate GANs?

## 5.1. IS(Inception Score)

- GAN의 성능을 평가하기 위해서는 다음과 같은 요소를 고려해야 한다.
    1. **Quality(Sharpness)**: 생성된 이미지의 품질이 좋아야 한다.
    2. **Diversity**: 다양한 이미지를 생성해야 한다. 만약 Mode collapse가 발생한다면, 한 가지 이미지만을 생성해낼 것이다.

- **IS(Inception Score)**: CV 모델인 Inception v3 분류기를 활용하여 GAN의 품질과 다양성을 나타내는 성능 지표
    - Quality(Sharpness)
$$
S = \exp \Big( \mathbb{E}_{x\sim p} \Big[ \int c(y|x) \log c(y|x) dy \Big] \Big)
$$
($c$: classifier, $y$: label, $x$: data, 품질이 좋을수록 $c(y|x)$의 엔트로피가 감소함.)
    - Diversity
$$
D = \exp \Big( -\mathbb{E}_{x \sim p} \Big[ \int c(y|x) \log c(y) dy \Big] \Big)
$$
($c(y) = \mathbb{E}_{x\sim p} [c(y|x)]$: classifier's marginal predictive distribution, 다양성이 좋을수록 $c(y)$의 엔트로피가 증가함.)
    - Inception Score: <u>점수가 높을수록 성능이 좋다.</u>
$$
IS = D \times S = \mathbb{E}_{x \sim p} [KL(p(y|x)||p(y))]
$$
(계산을 통해 식을 쉽게 유도할 수 있음.)

- IS의 문제점: Inception v3 모델에 의존한다는 것이 가장 큰 문제점이다.
    1. Inception v3 모델의 예측이 잘못된 경우 IS 점수가 떨어질 수 있다.
    2. 만약 Inception v3 모델에 존재하지 않는 분류인 경우 IS 점수가 크게 떨어질 수 있다.

## 5.2. FID(Frechet Inception Distance)

- **FID(Frechet Inception Distance)**: Inception v3 모델의 final hidden features를 활용하여 GAN의 성능을 평가하는 지표
    - FID는 <u>점수가 낮을수록 성능이 좋다.</u> FID의 첫 번째 항은 Quality(품질)의 의미를 지니고, FID의 두 번째 항은 Diversity(다양성)의 의미를 지닌다.
$$
FID = ||\mu_{T}-\mu_{G}||^{2} + Tr(\Sigma_{T} + \Sigma_{G} - 2(\Sigma_{T}\Sigma_{G})^{1/2})
$$

**[참고 자료]**
1. GAN 논문: https://arxiv.org/abs/1406.2661
2. GAN 논문 리뷰 유튜브 1: https://www.youtube.com/watch?v=AVvlDmhHgC4&t=1611s
3. GAN 논문 리뷰 유튜브 2: https://www.youtube.com/watch?v=odpjk7_tGY0&t=1647s
4. KL divergence 블로그: https://angeloyeo.github.io/2020/10/27/KL_divergence.html
5. KL divergence 위키백과: https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence
6. Cross Entropy minimum StackExchange: https://stats.stackexchange.com/questions/488032/minimum-value-of-cross-entropy-cost-function
7. Gibb's inequality 위키백과: https://en.wikipedia.org/wiki/Gibbs%27_inequality
8. Jensen-Shannon divegence 블로그: https://aigong.tistory.com/66
9. Jensen-Shannon divergence 위키백과: https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence
10. GAN 저자 Tutorial: https://arxiv.org/abs/1701.00160
11. (4) 생성 모델의 평가, wikidocs: https://wikidocs.net/149481
12. Cornell University CS 6785 lecture note: https://canvas.cornell.edu/courses/27332/assignments/syllabus